In [15]:
import os
import pandas as pd
from textblob import TextBlob
from dotenv import load_dotenv
import mysql.connector
import sqlalchemy
from sqlalchemy import update, MetaData, Table
load_dotenv()

True

In [2]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}/{os.getenv("DB_NAME")}')


In [3]:
df = pd.read_sql_table("Videos", engine)
print(df)

      id  influencer_id                                          url  \
0      1              4  https://www.youtube.com/watch?v=bXBMv6j28-s   
1      2              4  https://www.youtube.com/watch?v=bXBMv6j28-s   
2      3              4  https://www.youtube.com/watch?v=bXBMv6j28-s   
3      4              4  https://www.youtube.com/watch?v=bXBMv6j28-s   
4      5              4  https://www.youtube.com/watch?v=bXBMv6j28-s   
..   ...            ...                                          ...   
189  190             20  https://www.youtube.com/watch?v=9D_6xgq6OhM   
190  191             20  https://www.youtube.com/watch?v=9D_6xgq6OhM   
191  192             20  https://www.youtube.com/watch?v=9D_6xgq6OhM   
192  193             20  https://www.youtube.com/watch?v=9D_6xgq6OhM   
193  194             20  https://www.youtube.com/watch?v=9D_6xgq6OhM   

                           title  \
0        SAO PAULO // OUT NOW 🇧🇷   
1        SAO PAULO // OUT NOW 🇧🇷   
2        SAO PAULO // OUT N

In [5]:
df.head()

,id,influencer_id,url,title,comment,sentiment_score
0,1,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Deaf people:💀<br>Blind people:😄😃,None
1,2,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,bro what is this💀💀,None
2,3,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Eyes In shock😮😵‍💫 but ears Mesmerized🎶👌💥,None
3,4,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,"We&#39;re all traumatized, thank you Abel 😄",None
4,5,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Bro WHAT is this just drop the album Abel😭🙏🏽,None


In [6]:


# Define a function to assign sentiment scores based on polarity
def assign_score(polarity):
    if polarity > 0:
        return round(5 + (5 * polarity))  # Scale from 6 to 10 for positive sentiments
    elif polarity < 0:
        return round(5 * polarity) + 5    # Scale from 1 to 4 for negative sentiments
    else:
        return 5                          # Neutral sentiment is scored as 5

# Apply sentiment analysis and assign scores
df['polarity'] = df['comment'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['sentiment_score'] = df['polarity'].apply(assign_score)

# Drop the 'polarity' column as it's no longer needed
df.drop('polarity', axis=1, inplace=True)

In [7]:
df.head()

,id,influencer_id,url,title,comment,sentiment_score
0,1,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Deaf people:💀<br>Blind people:😄😃,5
1,2,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,bro what is this💀💀,5
2,3,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Eyes In shock😮😵‍💫 but ears Mesmerized🎶👌💥,5
3,4,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,"We&#39;re all traumatized, thank you Abel 😄",5
4,5,4,https://www.youtube.com/watch?v=bXBMv6j28-s,SAO PAULO // OUT NOW 🇧🇷,Bro WHAT is this just drop the album Abel😭🙏🏽,5


In [ ]:
metadata = MetaData()  # No need for 'bind' in SQLAlchemy 2.0
metadata.reflect(engine)  # Reflect the existing database schema

# Get reference to the 'Videos' table
videos_table = Table('Videos', metadata, autoload_with=engine)

# Establish a connection to execute updates
with engine.begin() as conn:  # Using `begin()` ensures that transaction is automatically committed or rolled back if an error occurs.
    for index, row in df.iterrows():
        # Debugging: Print out which row is being updated.
        print(f"Updating id {row['id']} with sentiment_score {row['sentiment_score']}")

        # Create an update statement for each row based on its 'id'
        stmt = (
            update(videos_table)
            .where(videos_table.c.id == row['id'])  # Match by 'id'
            .values(sentiment_score=row['sentiment_score'])  # Update only 'sentiment_score'
        )
        # Execute the update statement
        conn.execute(stmt)

print("Sentiment analysis complete and sentiment scores updated in the database.")


In [18]:
df2 = pd.read_sql_table("Videos", engine)
df2.head()

,id,sentiment_score
0,1,5
1,2,5
2,3,5
3,4,5
4,5,5
